In [ ]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder

In [ ]:
df = pd.read_csv("Crop_recommendation.csv")
df.head()

['rice' 'maize' 'chickpea' 'kidneybeans' 'pigeonpeas' 'mothbeans'
 'mungbean' 'blackgram' 'lentil' 'pomegranate' 'banana' 'mango' 'grapes'
 'watermelon' 'muskmelon' 'apple' 'orange' 'papaya' 'coconut' 'cotton'
 'jute' 'coffee']


In [ ]:

labels = df.label.unique()
print(labels)


def to_label(i):
  return labels[i]

In [1]:
# Modify the dataset by adding additional features

land_area = {
    'rice': '0.5–2 ha',
    'maize': '0.5–2 ha',
    'chickpea': '0.25–1 ha',
    'kidneybeans': '0.25–1 ha',
    'pigeonpeas': '0.25–1 ha',
    'mothbeans': '0.25–1 ha',
    'mungbean': '0.25–1 ha',
    'blackgram': '0.25–1 ha',
    'lentil': '0.25–1 ha',
    'pomegranate': '1–5 ha',
    'banana': '1–5 ha',
    'mango': '2–10 ha',
    'grapes': '1–5 ha',
    'watermelon': '0.5–2 ha',
    'muskmelon': '0.5–2 ha',
    'apple': '2–10 ha',
    'orange': '2–10 ha',
    'papaya': '1–5 ha',
    'coconut': '2–10 ha',
    'cotton': '2–10 ha',
    'jute': '2–10 ha',
    'coffee': '2–10 ha'
}
water_need = {
    'rice': 'high',
    'banana': 'high',
    'watermelon': 'high',
    'muskmelon': 'high',
    'grapes': 'high',
    'maize': 'medium',
    'cotton': 'medium',
    'coffee': 'medium',
    'papaya': 'medium',
    'orange': 'medium',
    'mango': 'medium',
    'apple': 'medium',
    'chickpea': 'low',
    'lentil': 'low',
    'mungbean': 'low',
    'blackgram': 'low',
    'kidneybeans': 'low',
    'pigeonpeas': 'low',
    'mothbeans': 'low',
    'pomegranate': 'low',
    'coconut': 'low',
    'jute': 'low'
}

# Mapping dasar
df['land_area'] = df['label'].map(land_area)
df['water_need'] = df['label'].map(water_need)

# Fungsi bantu untuk parsing min–max ha
def parse_area(area_str):
    print(area_str)
    if '–' in area_str:  # range
        parts = area_str.replace(' ha','').split('–')
        return float(parts[0]), float(parts[1])
    elif '≥' in area_str:  # minimal
        val = float(area_str.replace('≥','').replace(' ha',''))
        return val, np.nan
    else:
        val = float(area_str.replace(' ha',''))
        return val, val

# Apply parsing ke kolom
df[['land_area_min','land_area_max']] = df['land_area'].apply(
    lambda x: pd.Series(parse_area(x))
)
df.drop(['land_area'], axis=1, inplace=True)

# Konversi water_need jadi numerik
water_need_level = {'rendah':1, 'sedang':2, 'high':3}
df['water_need_level'] = df['water_need'].map(water_need_level)

df.drop(['water_need'], axis=1, inplace=True)
df.to_csv('output.csv', index=False)

NameError: name 'df' is not defined

In [ ]:

X = df.drop(["label"], axis=1)
y = pd.DataFrame(df.label)

In [ ]:
# Encode the label feature

oh_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
oh_cols = pd.DataFrame(oh_encoder.fit_transform(y))

# Return the index
oh_cols.index = y.index

y = oh_cols

In [ ]:
# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
# Start to predict

from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor()
model.fit(X_train, y_train)

predictions = model.predict(X_val)

accuracy = accuracy_score(y_val, predictions)

print("accuracy: ", accuracy)
print("predictions: ", predictions)


predictions = np.where(predictions)[1].tolist()

predictions = list(map(to_label, predictions))
print(predictions)